In [29]:
import os
import csv
from pymongo import MongoClient
from pprint import pprint

# Credentials

In [2]:
pw_file = 'credentials/mongo_pw.txt'
if os.path.exists(pw_file): 
    with open(pw_file, 'r') as f:
        pub_ip, mongo_usr, mongo_usr_pw = f.readline().strip().split(', ')

# Connect to MongoDB

In [3]:
# connect to ec2 mongo client
client = MongoClient('{0}:27017'.format(pub_ip))

# get reference to  resume_db
db = client['github_db']

# authenticate user for database
db.authenticate(mongo_usr, mongo_usr_pw)

True

# Show MongoDB collections

In [4]:
def show_collections():
    '''
    IN: None
    RETURN: Printed output of collections and collection counts
    '''
    for name in sorted(db.collection_names()):
        print('Documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

In [5]:
show_collections()

Documents in "git_repos_contributors" collection: 1013
Documents in "git_repos_docs" collection: 54031
Documents in "git_repos_meta" collection: 1011
Documents in "git_repos_subscribers" collection: 1013
Documents in "git_users_followers" collection: 233
Documents in "git_users_following" collection: 233
Documents in "git_users_meta" collection: 233


In [31]:
cursor = db['git_repos_docs'].find()

for i in cursor[:1]:
    fields = list(i.keys())
fields

['repo_fullname',
 'repo_owner_login',
 'repo_name',
 'file_name',
 'repo_id',
 'file_contents',
 'doc_name',
 '_id',
 'file_extension']

In [39]:
cursor = db['git_repos_docs'].find()[:10]
with open('test.csv', 'w') as outfile:
    writer = csv.DictWriter(outfile)
    writer.writeheader()
    for x in cursor:
        a = x.values()
        writer.writerow(a)

TypeError: __init__() missing 1 required positional argument: 'fieldnames'

In [34]:
cursor = db['git_repos_docs'].find()[:10]
output = csv.writer(open('some.csv', 'wt')) # writng in this file

for items in cursor: # first 11 entries
    a = list(items['file_name']) # collections are importent as dictionary and I am making them as list
    tt = list()
    for chiz in a:
        if chiz is not None:
            tt.append(chiz.encode('ascii', 'ignore')) #encoding
        else:
            tt.append("none")
    output.writerow(tt)